<h1><font color='blue'> 8E and 8F: Finding the Probability P(Y==1|X)</font></h1>

<h2><font color='Geen'> 8E: Implementing Decision Function of SVM RBF Kernel</font></h2>

<font face=' Comic Sans MS' size=3>After we train a kernel SVM model, we will be getting support vectors and their corresponsing coefficients $\alpha_{i}$

Check the documentation for better understanding of these attributes: 

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
<img src='https://i.imgur.com/K11msU4.png' width=500>

As a part of this assignment you will be implementing the ```decision_function()``` of kernel SVM, here decision_function() means based on the value return by ```decision_function()``` model will classify the data point either as positive or negative

Ex 1: In logistic regression After traning the models with the optimal weights $w$ we get, we will find the value $\frac{1}{1+\exp(-(wx+b))}$, if this value comes out to be < 0.5 we will mark it as negative class, else its positive class

Ex 2: In Linear SVM After traning the models with the optimal weights $w$ we get, we will find the value of $sign(wx+b)$, if this value comes out to be -ve we will mark it as negative class, else its positive class.

Similarly in Kernel SVM After traning the models with the coefficients $\alpha_{i}$ we get, we will find the value of 
$sign(\sum_{i=1}^{n}(y_{i}\alpha_{i}K(x_{i},x_{q})) + intercept)$, here $K(x_{i},x_{q})$ is the RBF kernel. If this value comes out to be -ve we will mark $x_{q}$ as negative class, else its positive class.

RBF kernel is defined as: $K(x_{i},x_{q})$ = $exp(-\gamma ||x_{i} - x_{q}||^2)$

For better understanding check this link: https://scikit-learn.org/stable/modules/svm.html#svm-mathematical-formulation
</font>

## Task E

> 1. Split the data into $X_{train}$(60), $X_{cv}$(20), $X_{test}$(20)

> 2. Train $SVC(gamma=0.001, C=100.)$ on the ($X_{train}$, $y_{train}$)

> 3. Get the decision boundry values $f_{cv}$ on the $X_{cv}$ data  i.e. ` `$f_{cv}$ ```= decision_function(```$X_{cv}$```)```  <font color='red'>you need to implement this decision_function()</font>

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
import numpy as np
from sklearn.svm import SVC

In [2]:
X, y = make_classification(n_samples=5000, n_features=5, n_redundant=2,
                           n_classes=2, weights=[0.7], class_sep=0.7, random_state=15)

### Pseudo code

clf = SVC(gamma=0.001, C=100.)<br>
clf.fit(Xtrain, ytrain)

<font color='green'>def</font> <font color='blue'>decision_function</font>(Xcv, ...): #use appropriate parameters <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='green'>for</font> a data point $x_q$ <font color='green'>in</font> Xcv: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color='grey'>#write code to implement $(\sum_{i=1}^{\text{all the support vectors}}(y_{i}\alpha_{i}K(x_{i},x_{q})) + intercept)$, here the values $y_i$, $\alpha_{i}$, and $intercept$ can be obtained from the trained model</font><br>
   <font color='green'>return</font> <font color='grey'><i># the decision_function output for all the data points in the Xcv</i></font>
    
fcv = decision_function(Xcv, ...)  <i># based on your requirement you can pass any other parameters </i>

<b>Note</b>: Make sure the values you get as fcv, should be equal to outputs of clf.decision_function(Xcv)


In [3]:
# you can write your code here
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,stratify = y,test_size = 0.2)
X_train,X_cv,Y_train,Y_cv = train_test_split(X_train,Y_train,stratify = Y_train,test_size = 0.25)

clf = SVC(gamma = 0.001,C = 100)
clf.fit(X_train,Y_train)
f_cv_master = clf.decision_function(X_cv)
intercept = clf.intercept_

In [4]:
SV = clf.support_vectors_
alpha_y_i = clf.dual_coef_
alpha_y_i = alpha_y_i.reshape(-1,1)
alpha_y_i.shape


(577, 1)

In [5]:
from scipy.spatial import distance
from tqdm import tqdm
import math

def decision_function(X_cv,SV,alpha_y_i,gamma,intercept):
    f_cv = []
    gamma = 0.001
    
    for x_q in tqdm(X_cv):
        sum1 = 0
        for k,x_i in enumerate(SV):
            sum1+= alpha_y_i[k]*(math.exp(-gamma*(distance.euclidean(x_q,SV[k])**2)))
        
        sum1+=intercept
        f_cv.append(round(sum1[0],8)) 
        
    return f_cv 

gamma = 0.001
f_cv_slave = decision_function(X_cv,SV,alpha_y_i,gamma,intercept)   

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 91.73it/s]


In [6]:
print(f_cv_master[:10])
print(f_cv_slave[:10])

[ 0.62865451 -1.32893918 -3.08056443 -4.11320069 -1.43270507 -3.71791514
 -1.59637489 -2.7666321  -5.32448941 -4.31882056]
[0.62865451, -1.32893918, -3.08056443, -4.11320069, -1.43270507, -3.71791514, -1.59637489, -2.7666321, -5.32448941, -4.31882056]


<h2><font color='Geen'> 8F: Implementing Platt Scaling to find P(Y==1|X)</font></h2>

Check this <a href='https://drive.google.com/open?id=133odBinMOIVb_rh_GQxxsyMRyW-Zts7a'>PDF</a>
<img src='https://i.imgur.com/CAMnVnh.png'>


## TASK F


> 4. Apply SGD algorithm with ($f_{cv}$, $y_{cv}$) and find the weight $W$ intercept $b$ ```Note: here our data is of one dimensional so we will have a one dimensional weight vector i.e W.shape (1,)``` 

> Note1: Don't forget to change the values of $y_{cv}$ as mentioned in the above image. you will calculate y+, y- based on data points in train data

> Note2: the Sklearn's SGD algorithm doesn't support the real valued outputs, you need to use the code that was done in the `'Logistic Regression with SGD and L2'` Assignment after modifying loss function, and use same parameters that used in that assignment.
<img src='https://i.imgur.com/zKYE9Oc.png'>
if Y[i] is 1, it will be replaced with y+ value else it will replaced with y- value

> 5. For a given data point from $X_{test}$, $P(Y=1|X) = \frac{1}{1+exp(-(W*f_{test}+ b))}$ where ` `$f_{test}$ ```= decision_function(```$X_{test}$```)```, W and b will be learned as metioned in the above step

__Note: in the above algorithm, the steps 2, 4 might need hyper parameter tuning, To reduce the complexity of the assignment we are excluding the hyerparameter tuning part, but intrested students can try that__


If any one wants to try other calibration algorithm istonic regression also please check these tutorials

1. http://fa.bianp.net/blog/tag/scikit-learn.html#fn:1

2. https://drive.google.com/open?id=1MzmA7QaP58RDzocB0RBmRiWfl7Co_VJ7

3. https://drive.google.com/open?id=133odBinMOIVb_rh_GQxxsyMRyW-Zts7a

4. https://stat.fandom.com/wiki/Isotonic_regression#Pool_Adjacent_Violators_Algorithm


In [16]:
import math

def sigmoid(z):
    sig = 1/(1+math.exp(-z))
    
    return sig

In [17]:
def gradient_dw(x,y,w,b,alpha,N):
    '''In this function, we will compute the gardient w.r.to w '''

    a = w[0]*x
    temp = x *(y-sigmoid(a + b))
    dw = temp-(w* (alpha/N))

    return dw

In [18]:
def gradient_db(x,y,w,b):
    '''In this function, we will compute gradient w.r.to b '''
    a = w*x
    
    db = y-sigmoid(a+b)
    return db

In [19]:
from tqdm import tqdm
def train(X_train,y_train,epochs,alpha,eta0):
    
    w = np.array([0])
    b = 0
    
    for i in tqdm(range(epochs)):
        for j in range(len(X_train)):
            dw = gradient_dw(X_train[j],y_train[j],w,b,alpha,len(X_train))
            db = gradient_db(X_train[j],y_train[j],w,b)
            
            w = w + eta0*dw
            b = b + eta0*db
            
    return w,b

In [20]:
Y_cv1 = []

for i in range(len(Y_cv)):
    Y_cv1.append(Y_cv[i])    

In [21]:
y_p = Y_cv1.count(1)
y_n = Y_cv1.count(0)

pos = float(y_p + 1)/(y_p+2)
neg = 1/float(y_n+2)

for i in range(len(Y_cv1)):
    if(Y_cv1[i]==1):
        Y_cv1[i] = pos
    else:
        Y_cv1[i] = neg
print(pos,neg)
Y_cv2 = np.array(Y_cv1)

0.9967213114754099 0.001430615164520744


In [22]:
alpha=0.0001
eta0=0.0001
N=len(Y_cv2)
epochs=50

w,b = train(Y_cv2,f_cv_slave,epochs,alpha,eta0)

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 114.45it/s]


In [23]:
print(w,b)

[1.50877492] -7.065664560900679


In [25]:
import math
f_test = decision_function(X_test,SV,alpha_y_i,gamma,intercept)
P_X_1 = [1/(1+math.exp(-(w*x + b))) for x in f_test]


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 92.95it/s]


In [26]:
print(P_X_1[:10])

[2.655744200366622e-05, 0.012780531936096448, 2.3849817855945682e-05, 3.347027523512707e-05, 0.387939916985917, 6.770887444003207e-06, 9.030394258307848e-05, 9.659566356423562e-06, 0.03677936537330718, 0.00030176277604018303]
